## Quick Start - Simple Prompt invoke with chain

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

# Define the language model
llm = ChatOpenAI(openai_api_key="")

# Define the output parser - Output like a string an not an object
output_parser = StrOutputParser()

# Now we obtain a string from invoke
chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith can generate code snippets and test cases automatically, allowing developers to quickly create tests for their code. This can save time and effort in writing test cases manually.\n\n2. Test Data Generation: Langsmith can generate test data for various scenarios, making it easier to test different edge cases and input combinations. This can help in improving test coverage and ensuring the robustness of the software.\n\n3. Code Analysis: Langsmith can analyze code for potential issues, such as bugs, vulnerabilities, or performance bottlenecks. By identifying these issues early on, developers can address them before they become bigger problems and impact testing efforts.\n\n4. Integration Testing: Langsmith can assist in setting up and running integration tests by providing code snippets and best practices for testing interactions between different components or modules of the software.\n\n5. Performance 

## Retrieval Chain

Using beautifulsoup4 for information retrival

WebBaseLoader è un metodo della libraria di LangChain che va ad estrarre il testo

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [6]:
print(docs)

[Document(page_content="\n\n\n\n\nLangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî is incredibly important for this phase.Debuggin

Now we need to index it into vectorspace

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

and create the embeddings

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now we have the data into the vectorspace and we can pass them to the LLM. The point is to pass just the relevant information to the model and not the entire database. That's why we're talking about **retrival**.

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

Let's test a prompt with context

In [15]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmixth help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

And compare with no context prompt

In [16]:
## TODO

In [13]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
import textwrap

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
output = response["answer"]

# Just a pretty print
wrapped_text = textwrap.fill(output, width=80)
print(wrapped_text)

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing developers to create datasets, run
tests on their LLM applications, upload test cases in bulk, create test cases on
the fly, export test cases from application traces, and run custom evaluations
to score test results. Additionally, LangSmith provides a comparison view to
track and diagnose regressions in test scores across multiple revisions of an
application, a playground environment for rapid iteration and experimentation,
and the ability to add runs as examples to datasets to expand test coverage on
real-world scenarios.
